# Compared with networkx

Based on : https://github.com/cwida/duckpgq-extension/pull/137

In [1]:
from connectedComponentsLabeling import connectedComponentsLabeler as ccl
from time import time
import pandas as pd
import networkx as nx
import numpy as np
import duckdb

In [2]:
def simulateDataFrame(numberOfEdges=100*1000,numberOfNodes=10*1000):
    return pd.DataFrame({'a':np.random.randint(0, high=numberOfNodes, size=numberOfEdges),
                         'b':np.random.randint(0, high=numberOfNodes, size=numberOfEdges)})

In [3]:
numberOfEdges=4000000
numberOfNodes=8000000

In [4]:
yoba=simulateDataFrame(numberOfEdges=numberOfEdges,numberOfNodes=numberOfNodes)
yoba.head()

,a,b
0,7122080,6839999
1,6478375,3583191
2,3930726,2642690
3,4882378,5194507
4,7399109,7786992


In [5]:
%%time 

startTime=time()

yoba_cc=ccl(yoba).getConnectedCompontents()

yobaTime=time()-startTime

CPU times: user 23 s, sys: 1.02 s, total: 24 s
Wall time: 24 s


In [6]:
yoba_cc.head()

,nodeId,componentId
0,0,4644612
1,1,1
2,3,715135
3,4,3
4,6,4150796


In [7]:
def calculate_cc(G: nx.Graph) -> pd.DataFrame:
    # works on undirected graph
    connected_components = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]
    # Prepare lists to hold the IDs and their corresponding componentId
    ids = []
    component_ids = []

    for component in connected_components:
        # Determine the componentId as the minimum node ID in the component
        component_id = min(component)

        # Add each node in the component to the lists
        for node in component:
            ids.append(int(node))
            component_ids.append(int(component_id))

    # Create a DataFrame from the lists
    wcc_df = pd.DataFrame({
        'id': ids,
        'componentId': component_ids
    })

    return wcc_df

In [8]:
G=nx.from_pandas_edgelist(yoba,source='a',target='b')

In [10]:
%%time

netX=calculate_cc(G)

CPU times: user 12.8 s, sys: 717 ms, total: 13.6 s
Wall time: 13.5 s


In [11]:
duckdb.sql('select count(distinct componentID) from netX')

┌─────────────────────────────┐
│ count(DISTINCT componentID) │
│            int64            │
├─────────────────────────────┤
│                     1058241 │
└─────────────────────────────┘

In [12]:
duckdb.sql('select count(distinct componentId) from yoba_cc')

┌─────────────────────────────┐
│ count(DISTINCT componentId) │
│            int64            │
├─────────────────────────────┤
│                     1058241 │
└─────────────────────────────┘